In [14]:
from itertools import count
from collections import defaultdict
from scipy.sparse import csr
import numpy as np
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm


def vectorize_dic(dic,ix=None,p=None,n=0,g=0):
    """
    dic -- dictionary of feature lists. Keys are the name of features
    ix -- index generator (default None)
    p -- dimension of featrure space (number of columns in the sparse matrix) (default None)
    """
    if ix==None:
        ix = dict()

    nz = n * g

    col_ix = np.empty(nz,dtype = int)

    i = 0
    for k,lis in dic.items():
        for t in range(len(lis)):
            ix[str(lis[t]) + str(k)] = ix.get(str(lis[t]) + str(k),0) + 1
            col_ix[i+t*g] = ix[str(lis[t]) + str(k)]
        i += 1

    row_ix = np.repeat(np.arange(0,n),g)
    data = np.ones(nz)
    if p == None:
        p = len(ix)

    ixx = np.where(col_ix < p)
    return csr.csr_matrix((data[ixx],(row_ix[ixx],col_ix[ixx])),shape=(n,p)),ix


def batcher(X_, y_=None, batch_size=-1):
    n_samples = X_.shape[0]

    if batch_size == -1:
        batch_size = n_samples
    if batch_size < 1:
       raise ValueError('Parameter batch_size={} is unsupported'.format(batch_size))

    for i in range(0, n_samples, batch_size):
        upper_bound = min(i + batch_size, n_samples)
        ret_x = X_[i:upper_bound]
        ret_y = None
        if y_ is not None:
            ret_y = y_[i:i + batch_size]
            yield (ret_x, ret_y)


cols = ['user','item','rating','timestamp']

train = pd.read_csv('data/ua.base',delimiter='\t',names = cols)
test = pd.read_csv('data/ua.test',delimiter='\t',names = cols)

x_train,ix = vectorize_dic({'users':train['user'].values,
                            'items':train['item'].values},n=len(train.index),g=2)


x_test,ix = vectorize_dic({'users':test['user'].values,
                           'items':test['item'].values},ix,x_train.shape[1],n=len(test.index),g=2)


print(x_train)
y_train = train['rating'].values
y_test = test['rating'].values

x_train = x_train.todense()
x_test = x_test.todense()

print(x_train)

print(x_train.shape)
print (x_test.shape)

  (0, 1)	2.0
  (1, 1)	1.0
  (1, 2)	1.0
  (2, 1)	1.0
  (2, 3)	1.0
  (3, 1)	1.0
  (3, 4)	1.0
  (4, 1)	1.0
  (4, 5)	1.0
  (5, 1)	1.0
  (5, 6)	1.0
  (6, 1)	1.0
  (6, 7)	1.0
  (7, 1)	1.0
  (7, 8)	1.0
  (8, 1)	1.0
  (8, 9)	1.0
  (9, 1)	1.0
  (9, 10)	1.0
  (10, 1)	1.0
  (10, 11)	1.0
  (11, 1)	1.0
  (11, 12)	1.0
  (12, 1)	1.0
  (12, 13)	1.0
  :	:
  (90557, 146)	1.0
  (90558, 51)	1.0
  (90558, 147)	1.0
  (90559, 96)	1.0
  (90559, 148)	1.0
  (90560, 44)	1.0
  (90560, 149)	1.0
  (90561, 38)	1.0
  (90561, 150)	1.0
  (90562, 90)	1.0
  (90562, 151)	1.0
  (90563, 133)	1.0
  (90563, 152)	1.0
  (90564, 38)	1.0
  (90564, 153)	1.0
  (90565, 115)	1.0
  (90565, 154)	1.0
  (90566, 71)	1.0
  (90566, 155)	1.0
  (90567, 41)	1.0
  (90567, 156)	1.0
  (90568, 46)	1.0
  (90568, 157)	1.0
  (90569, 4)	1.0
  (90569, 158)	1.0
[[ 0.  2.  0. ...,  0.  0.  0.]
 [ 0.  1.  1. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  

In [15]:
n,p = x_train.shape

k = 10

w0 = tf.Variable(tf.zeros([1]))
w = tf.Variable(tf.zeros([p]))
v = tf.Variable(tf.random_normal([p, k], mean=0, stddev=0.01))

x = tf.placeholder('float', [None, p])
y = tf.placeholder('float', [None, 1])

linear_terms = tf.add(w0, tf.reduce_sum(
    tf.multiply(x, w), axis=1, keep_dims=True))
pair_terms = 0.5 * tf.subtract(tf.reduce_sum(tf.pow(tf.matmul(x, v), 2), 1),
                               tf.reduce_sum(tf.matmul(tf.pow(x, 2), tf.pow(v, 2)), 1))
y_hat = tf.add(linear_terms, pair_terms)

lambda_w = tf.constant(0.001, name='lambda_w')
lambda_v = tf.constant(0.001, name='lambda_v')

l2_norm = tf.add(tf.reduce_sum(tf.multiply(
    lambda_w, tf.pow(w, 2))), tf.reduce_sum(tf.multiply(lambda_v, tf.pow(v, 2))))

error = tf.reduce_mean(tf.square(y - y_hat))
loss = tf.add(error, l2_norm)

train_op = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

In [21]:
epochs = 10
batch_size = 100

init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    
    errors = []
    
    for epoch in tqdm(range(epochs), unit='epoch'):
        for bx, by in batcher(x_train, y_train, batch_size):
            _, t = sess.run([train_op, loss], feed_dict={x: bx.reshape(-1, p), y:by.reshape(-1, 1)})
#             print(t)
            
        
        for bX, bY in batcher(x_test, y_test):
            errors.append(sess.run(error, feed_dict={x: bX.reshape(-1, p), y: bY.reshape(-1, 1)}))
            print(errors)
        RMSE = np.sqrt(np.array(errors).mean())
        print (RMSE)


A Jupyter Widget

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/muzhen/anaconda3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/Users/muzhen/anaconda3/lib/python3.6/site-packages/tqdm/_monitor.py", line 63, in run
    for instance in self.tqdm_cls._instances:
  File "/Users/muzhen/anaconda3/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



[1.0934601]
1.04569
[1.0934601, 1.0959227]
1.04628
[1.0934601, 1.0959227, 1.0982112]
1.04684
[1.0934601, 1.0959227, 1.0982112, 1.1010331]
1.04745
[1.0934601, 1.0959227, 1.0982112, 1.1010331, 1.1029675]
1.04801
[1.0934601, 1.0959227, 1.0982112, 1.1010331, 1.1029675, 1.1043241]
1.04848
[1.0934601, 1.0959227, 1.0982112, 1.1010331, 1.1029675, 1.1043241, 1.1051801]
1.04888
[1.0934601, 1.0959227, 1.0982112, 1.1010331, 1.1029675, 1.1043241, 1.1051801, 1.1056724]
1.04921
[1.0934601, 1.0959227, 1.0982112, 1.1010331, 1.1029675, 1.1043241, 1.1051801, 1.1056724, 1.1063873]
1.04951
[1.0934601, 1.0959227, 1.0982112, 1.1010331, 1.1029675, 1.1043241, 1.1051801, 1.1056724, 1.1063873, 1.1072685]
1.04978

